# Flow and transport in 3D fractured medium

Description: This notebook presents a weakly coupled flow and transport problem reported in the paper *PorePy: An Open-Source Simulation Tool for Flow and Transport in Deformable Fractured Rocks*, by
Eirik Keilegavlen, Alessio Fumagalli, Runar Berge, Ivar Stefansson, Inga Berre. See arXiv:1712:00460 for details. The code in the notebook was that used to produce figure 5 in the paper, and if ran on a separate system, (almost) the same results should result - factors such as grid generation, linear solvers etc. may produce minor differences.

The notebook doubles as an introduciton to PorePy for multi-physics, and is therefore rather long. To navigate quickly through the procedure, the main steps are: i) Create a mesh, ii) assign parameters for both flow and transport problems, iii) set up appropriate solvers, iv) discretize and solve.

## Preliminaries
To run this, you need to have PorePy installed and set up with Gmsh. Depending on the computer, the simulations can be somewhat memory and time consuming. To run on a subset of the fractures, use parameters for the grid generation function below.

Also, it is recommended to install pyamg to speed up preconditioning for the pressure solver.

## Imports
The first step is to import all dependencies

In [1]:
# External modules
import numpy as np
import pickle
import scipy.sparse as sps
import logging
import sys

# For plotting
from IPython.display import HTML, display

# import porepy
import porepy as pp


/home/eke001/Dropbox/workspace/python/porepy/src/porepy/grids/partition.py:18: UserWarning: Could not import pymetis. Some functions will not work as    intended
  intended')


## Logger
Next, set up a logger to monitor the progress.

The syntax here is a bit confusing: The levels of the logging module in Python has names indicating that the program may break down (DEBUG, INFO, WARNING, ERROR, CRITICAL). We use them instead to set the level of verbosity, with DEBUG corresponding to 'print everything', and CRITICAL being almost silent. The tuning of these levels, and achieving consistency between modules, is still a work in progress.

In [2]:
# Define
root = logging.getLogger()
root.setLevel(logging.WARN)

# Only add handler if none is assigned. Without this, a new handler will be added every time this cell is ran.
if not root.hasHandlers():
    # Print to sys.stdout
    ch = logging.StreamHandler(sys.stdout)
    ch.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(message)s')
    ch.setFormatter(formatter)

    root.addHandler(ch)

## Grid generation
The below function creates a 3D fracture etwork from a 2D outcrop. The extrusion produc vertical fractures only - more advanced options are available, but we have no geological basis for using them here.

In [3]:
def create_grid(num_fractures=None):
    """ Create a grid from an outcrop. 
    
    To adjust the number of fractures to include, set num_fractures < 69 (maximum)
    """
    # Default is to create all fractures
    if num_fractures == None:
        num_fractures = 66

    # Fix the random seed, so that the produced network is reproducible
    np.random.seed(7)
    # Read points in outcrop from line, together with their connections.
    # The file 'barely-connected_4pp.csv' is stochastically generated based 
    # on data from a real outcrop. Noteworthy, it contains several
    # Y-intersections, which are preserved in extrusion.
    # For more details, see tutorial on fracture extrusion.
    pt, edges = pp.importer.lines_from_csv('barely-connected_4pp.csv')
    
    # Extrude fractures.
    # Assign same family name to all fractures, and let them have zero incline.
    # This is the place to modify if you want fractures with different 
    # properties, e.g. if they have different geological origins.
    frac_family = np.zeros(edges.shape[1], dtype=np.int)
    fractures = pp.extrusion.fractures_from_outcrop(pt, edges, family=frac_family,
                                                    family_std_incline=[0.], tol=1e-5)

    # Pick out the subset of fractures (or all) to be included in the grid
    i = np.arange(num_fractures)
    f = [fractures[fi] for fi in i]
    # Create fracture network representation of the grid
    network = pp.FractureNetwork(f)
    # Assign tolerance used for computations. This is sometimes a critical 
    # parameter, it roughly defines how close to geometric objcets can be
    # before they are considered identical (in case of two points), crossing
    # (in case of two lines) etc.
    # For general networks, it may be necessary to experiment a bit with this
    # parameter to generate a reasonable mesh
    network.tol = 1e-5
    
    # Export the network itself (no grid yet) as a vtu file, accesible by Paraview
    network.to_vtk('network.vtu')

    # Generate the mixed-dimensional grid.
    # The mesh size is controlled by two parameters: mesh_size_frac is the target 
    # mesh size, while mesh_size_min is the minimal mesh size specified in the Gmsh
    # configuration file. To which degree Gmsh adhers to this parameter
    # varies from one case to the next.
    # The parameter ensure_matching_face_cells=False is used to avoid an error
    # message when matching cells with faces in the various dimensions
    # The origin of the trouble seems to be in Gmsh (has been reported).
    # We have never experienced any trouble with using this workaround, but
    # some caution is still due.
    gb = meshing.simplex_grid(network=network, mesh_size_frac=0.1, mesh_size_min=0.01,
                              verbose=1,ensure_matching_face_cell=False)
    
    # gb is now a GridBucket, that is, a mixed-dimensional grid.
    # Technically, this is implemented as a graph, with each node representing
    # a grid (the matrix grid, a fracture, the intersection between fractures).
    
    # Rescale the domain from a meter to hundred meter scale, just to get more
    # realistic numbers.
    for g, _ in gb:
        g.nodes *= 100 * pp.METER
    gb.compute_geometry()
    
    return gb

## Problem setup: Parameter specification and solvers
To set up the flow and transport problems, we will use, respectively, an EllipticModel and a ParabolicModel. These are solvers dedicated to solving elliptic equations, e.g. the incompressible pressure equation, and advection-diffusion problems. The models automatically sets up simple discretization schemes, and tries to design decent linear solvers for the resulting systems of equations.

### Pressure solver
The first step is to provide simulation data. Simulation parameters are stored as part of the GridBucket, but accessing this can be somewhat cumbersome. To assist the assignment, and also to provide a reasonable way of setting default parameters, each of the models (pre-defined solvers), are accompanied by a DataAssigner. This can be used directly to define a simulaiton with default parameters, or modified as desired. Below, we give an example for the pressure equation.

In [4]:
# The units module contains various physical constants
# Set a relative high matrix permeability, this would correspond to
# a quite high density of upscaled fractures.
matrix_perm = 5 * pp.DARCY

class MatrixDomainPressure(pp.EllipticDataAssigner):
    """ Set data for the 3D domain (matrix) in the pressure equation.
    
    Fields that are not assigned here, will have the default values 
    prescribed in EllipticData (which again may point further to defaults
    in the Parameter class).
    """
    def __init__(self, g, dat):
        self.dim = g.dim
        pp.EllipticDataAssigner.__init__(self, g, dat)

    # The permeability is set as isotropic and homogeneous.
    # Both anisotropic and heterogeneous fields are easily accomodated
    # by changing this line.
    def permeability(self):
        kxx = matrix_perm * np.ones(self.grid().num_cells)
        return pp.SecondOrderTensor(self.dim, kxx)

    # Boundary conditions: By default, Neumann conditions are assigned.
    # Overwride these for two or the faces to set up flow along the y-axis.
    # Note that this function only sets the type of boundary conditions,
    # the exact values are set in the next function.
    def bc(self):
        g = self.grid()
        # This will pick out faces with the maximum or minimum y coordinate
        # We benefit from knowing that the simulation domain is a box alligned
        # with the axes, if not, picking out faces would have been more cumbersome.
        p_face = pp.params.bc.face_on_side(g, ['ymin', 'ymax'])
        p_face = np.hstack((p_face[0], p_face[1]))
        dir_str = p_face.size * ['dir']
        return pp.BoundaryCondition(g, faces=p_face, cond=dir_str)

    # Boundary values are set by specifying the value per face.
    # If none are specified, homogeneous conditions are assumed.
    # The reason for splitting types and values for boundary conditions is that 
    # types are needed at the time of discretization, while values are not imposed
    # until assembley of linear system. 
    def bc_val(self):
        g = self.grid()
        y_min = pp.params.bc.face_on_side(g, 'ymin')
        y_max = pp.params.bc.face_on_side(g, 'ymax')

        bnd = np.zeros(g.num_faces)
        bnd[y_min] = 0
        bnd[y_max] = 50 * pp.BAR
        return bnd

    def porosity(self):
        return 0.01 * np.ones(self.grid().num_cells)

Next, define parameters for the fractures, and their intersections. Most of the values are inherited from the matrix domain, but some are overwritten:

In [5]:
class FractureDomainPressure(MatrixDomainPressure):
    def __init__(self, g, dat):
        MatrixDomainPressure.__init__(self, g, dat)

    # Define an aperture of 1cm.
    # Note that the power is set according to the dimension, thus an 
    # intersection will have a transverse area of 1 cm squared.
    def aperture(self):
        return np.power(1 * pp.CENTI * pp.METER, 3 - self.grid().dim)

    # Set a permeability of 10^-4, corresponding to the square of the aperture.
    # The flow will again be scaled with the aperture inside the discretization,
    # thus we in effect apply the so-called cubic law.
    def permeability(self):
        kxx = np.ones(self.grid().num_cells)*np.power(1e-4, self.grid().dim<3)
        return pp.SecondOrderTensor(3, kxx)

    # A lot can be said about the boundary conditions at fracture tips.
    # We simply set a Neumann condition. Since this is the default, we 
    # need not define anything, but if we had not created the function, we would
    # have inherited from the matrix domain. This might have been okay, as the
    # fractures do not extend to the domain boundary, but the present version
    # is cleaner and safer.
    def bc(self):
        return pp.BoundaryCondition(self.grid())

    # No-flow conditions at fracture tips
    def bc_val(self):
        return np.zeros(self.grid().num_faces)

    def porosity(self):
        return np.ones(self.grid().num_cells)

Having defined parameter classes for all geometric objects, assigning the data is easy: Simply loop over the GridBucket, and choose DataAssigner according to the grid dimension.

If the data was heterogeneous, for instance some fractures are blocking, this would have required a few more steps. First, an extra DataAssigner is needed (BlockingFractureDomainPressure?), which would inherit from FractureDomainPressure, and likely override the permeability() function. Second, we need a way to differ between the individual fracture objects. Fracture grids will have an attribute frac_num, which reflects the order of the fractures when they were fed into the FractureNetwork, and further into the meshing algorithm. This can be used to identify the individual fractures.
    

In [6]:
# Define method to assign parameters to all nodes in the GridBucket
def assign_pressure_data(gb):
    # Prepare the model to recieve problems to be solved.
    gb.add_node_props(['problem'])
    # Loop over the grid bucket. This loops over the grid, and the
    # data field associated with the grid. The latter can be used to stored
    # pretty much anything, among them the problem definition.
    for g, d in gb:
        # The 3D domain is assigned Matrix properties
        if g.dim == 3:
            d['problem'] = MatrixDomainPressure(g, d)
        # All others get fracture properties.
        elif g.dim < 3:
            d['problem'] = FractureDomainPressure(g, d)
    # Assign coupling discharge
    gb.add_edge_props(['param'])
    for e, d in gb.edges():
        g_h = gb.nodes_of_edge(e)[1]
        d['param'] = pp.Parameters(g_h)


Finally, define the pressure solver. It would have been equally natural to let assign_pressure_data() be incorporated into __init__, but we pulled it out here to give it the proper attention:

In [7]:
class PressureSolver(pp.EllipticModel):
    """ Solver for the pressure equation.
    
    No methods are specified here, thus all functionality is inherited from 
    EllipticModel. 
    """
    def __init__(self, gb, solver_data):
        assign_pressure_data(gb)
        pp.EllipticModel.__init__(self, gb)


## Transport discretization
So far, we have defined a solver for the pressure equation only. Next, do the same for the transport solver. The steps are the same: To assign parameters, we will rely upon the ParabolicDataAssigner for default values, and overwrite values when appropriate. Then define a solver, and we are done.

In [8]:
class MatrixDomainTransport(pp.ParabolicDataAssigner):
    def __init__(self, g, dat):
        self.dim = g.dim
        pp.ParabolicDataAssigner.__init__(self, g, dat)

    # Thermal diffusivity of granite
    def diffusivity(self):
        kxx = 3 * np.ones(self.grid().num_cells)
        return pp.SecondOrderTensor(self.grid().dim, kxx)

    # Boundary conditions are set similar to the pressure equation:
    # Impose a gradient in the y-direction, and insulate the other boundaries.
    def bc(self):
        g = self.grid()
        p_face = pp.params.bc.face_on_side(g, ['ymin', 'ymax'])
        p_face = np.hstack((p_face[0], p_face[1]))
        dir_str = p_face.size * ['dir']
        return pp.BoundaryCondition(g, faces=p_face, cond=dir_str)

    # Boundary values are set to 0 and 1. Can easily be changed by scaling.
    def bc_val(self, t):
        g = self.grid()
        y_min = pp.params.bc.face_on_side(g, 'ymin')
        y_max = pp.params.bc.face_on_side(g, 'ymax')

        bnd = np.zeros(g.num_faces)
        bnd[y_min] = 0
        bnd[y_max] = 1
        return bnd

    # For implementation reasons, domains of all dimensions have an apperture.
    # In fractures and fracture intersections, this is used to scale cell volumes to a 
    # Nd value. In 3D, simply set the aperture to 1; this corresponds to a unit scaling.
    def aperture(self):
        return np.ones(self.grid().num_cells)

    def porosity(self):
        return 0.01 * np.ones(self.grid().num_cells)

    # Rock density corresponding (approximate) average values for granite. Values from Wikipedia.
    def rock_density(self):
        return 2.7 * pp.KILO * pp.KILOGRAM / pp.METER**3

    # Rough estiamet of water density
    def fluid_density(self):
        return 1.0 * pp.KILO * pp.KILOGRAM / pp.METER**3

    # Rock heat capasity for granite
    def rock_specific_heat(self):
        return 790. / pp.KILOGRAM

    # Heat capasity corresponds to water in standard conditions
    def fluid_specific_heat(self):
        return 4185. / (pp.KILOGRAM)


class FractureDomainTransport(MatrixDomainTransport):
    """ The FractureDomain inherits from Matrix domain. The difference is in the 
    aperture, that has the same value as for the flow problem.
    
    
    """
    def __init__(self, g, dat):
        MatrixDomainTransport.__init__(self, g, dat)

    def aperture(self):
        return np.power(1 * pp.CENTI * pp.METER, 3 - self.grid().dim)

    def bc(self):
        # Overwrite MatrixDomain's bc, we're using only Neumann
        return pp.params.bc.BoundaryCondition(self.grid())

    def bc_val(self, t):
        return np.zeros(self.grid().num_faces)

    def porosity(self):
        return np.ones(self.grid().num_cells)


Now, define the transport solver. The most noteworthy point here is the solver() method, which returns an implicit time stepper with a linear solver tailored for large, advection-dominated problems. Also, this is the place where the end time for simulation and time step size is set.

In [33]:
class TransportSolver(pp.ParabolicModel):
    def __init__(self, gb):
        """ Here the data assignment is located in __init__,
        as indicated in the pressure solver
        """
        gb.add_node_props(['transport_data'])
        for g, d in gb:
            if g.dim == 3:
                d['transport_data'] = MatrixDomainTransport(g, d)
            elif g.dim < 3:
                d['transport_data'] = FractureDomainTransport(g, d)

        pp.ParabolicModel.__init__(self, gb)

    # The spatial discretization returns both advective and diffusive
    # discretization. If we were to solve a pure advective problem, we
    # would simply return only self.advective_disc()
    def space_disc(self):
        return self.advective_disc(), self.diffusive_disc()
    
    def solver(self):
        # Create an ad hoc implicit time stepper that inherits from the
        # standard Implicit class (backward Euler approach).
        # The only change we do is to let the linear solver use
        # GMRES, preconditioned with ILU for large problems.
        # The definition of 'Large' is somewhat arbitrary; the point is 
        # to turn to an iteratie solver for systems large enough that a
        # direct solver is inefficient, or consumes too much memory.
        # What that means is very much a user specific parameter; 
        # the value prescribed here should be conservative in most cases.
        class ImplicitSolverGmres(pp.Implicit):

            def __init__(self, model):
                pp.Implicit.__init__(self, model)

            def step(self):
                from porepy.numerics.linalg.linsolve import Factory as LSFactory
                A = self.lhs
                b = self.rhs

                ls = LSFactory()
                if b.size < 40000:
                    self.p = ls.direct(A, b)
                    return self.p
                else:
                    # Note use of ILU here: This is appropriate for 
                    # advection dominated problems, where most of the action
                    # is local. For conduction dominated problems, a better
                    # approach would be to use elliptic solvers, e.g. a
                    # algebraic multigrid approach
                    precond = ls.ilu(A)
                    slv = ls.gmres(self.lhs)
                    # The use of callback here will log information on the
                    # iteration procedure.
                    # Parameters for maxiter, restart etc. are somewhat random.
                    # In practice, the iterations converges quite fast for this 
                    # problem.
                    self.p, info = slv(self.rhs, M=precond, callback=True,
                               maxiter=10000, restart=1500, tol=1e-8)
                    return self.p
        return ImplicitSolverGmres(self)

    def end_time(self):
        return 10 * pp.YEAR

    def time_step(self):
        return self.end_time()/10


# Set up solvers, then solve
We are finally ready to define our solver objects and solve for flow and temperature. With all parameters defined, this is a relatively simple code:

In [28]:
# Define computaitonal mesh (mixed-dimensional)
# You may also get a warning here, related to inconsistency between cells and faces.
# This is caused by what appears to be a bug in gmsh (we have reported),
# that shows up for small and skewed cells and faces.
# We have not experienced any practical implications of this.
mesh = create_grid()

# Create an exporter object, and dump the grid
# NOTE: The export, and particular representing the grid in a format 
# amenable for the unstructured data format in vtu is time consuming
# for a pure python code. If numba is available (conda install numba),
# a just in time compiler will be invoked, and the whole process
# speeds up dramatically. Do install.
mesh_exp = pp.Exporter(mesh, 'grid')
mesh_exp.write_vtk()

/home/eke001/anaconda3/envs/porepy/lib/python3.5/site-packages/vtk/util/numpy_support.py:134: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


Construct mesh
Use existing decomposition
Minimal distance between points encountered is 0.004273543932074861
Gmsh processed file successfully


Grid creation completed. Elapsed time 0.1820986270904541


Created 1 3-d grids with 10989 cells
Created 3 2-d grids with 1100 cells


Done. Elapsed time 1.0201530456542969
Assemble in bucket
Done. Elapsed time 0.026569604873657227
Compute geometry
Done. Elapsed time 0.047032833099365234
Split fractures
Done. Elapsed time 0.7815241813659668
Mesh construction completed. Total time 1.8778269290924072


The resulting grid looks like this, after some manipulation in Paraview

In [29]:
display(HTML("<table><tr><td><img src='fig/g0.png'></td><td><img src='fig/g1.png'></td></tr></table>"))

,


### Pressure solve

In [30]:
# Define pressure solver for the given grid.
# This will assign parameters, using the above classes
pressure_solver = PressureSolver(mesh, None)

# Solve the pressure equation.
#
# TECHNICAL INFORMATION:
# This discretizes the equation using a two-point flux approximation 
# (unless the discretization method is overwritten), assembles 
# the linear system and right hand side contributions from sources
# and boundary conditions. The linear system is then solved, hopefully
# with a somewhat efficient solver. Exactly how this should be defined is 
# not entirely clear at the moment (the linear system has saddle-point
# characteristics, which makes solution a bit challenging). What we do
# for now is to apply a direct solver for small systems (specified by the
# parameter max_direct). For large problem, a domain decomposition approach
# is applied for the rock matrix and all fractures and intersection. For
# large sub-problems (in practice the matrix), an AMG solver is invoked
# if available, if not, ILU is the fallback.
# It is strongly recommended to install pyamg before solving large problems.
#
# UPPSHOT:
# For large problems, this may take some time, and you may want to monitor memory usage.

pressure_solver.solve(callback=True)  # Use callback to monitor progression

# The pressure solution is stored as a single vector, considering all
# geometric objects (rock matrix, fractures, intersections) at the same time
# Split into the different components in the computational mesh.
pressure_solver.split()

# Store the solution in the data-part of the computational grid.
# This saves the solution to vtk.
pressure_solver.save('solution')

# Derive fluxes from the computed pressures
pp.fvutils.compute_discharges(mesh, p_name='solution')

2018-03-19 14:37:21,083 - porepy.numerics.elliptic - Solve elliptic model
2018-03-19 14:37:21,084 - porepy.numerics.elliptic - Discretize
2018-03-19 14:37:21,165 - porepy.numerics.elliptic - Done. Elapsed time 0.08112049102783203
2018-03-19 14:37:21,165 - porepy.numerics.elliptic - Solve linear system using direct solver
2018-03-19 14:37:21,352 - porepy.numerics.elliptic - Done. Elapsed time 0.18686413764953613


/home/eke001/anaconda3/envs/porepy/lib/python3.5/site-packages/vtk/util/numpy_support.py:134: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


Here is what the pressure solution looks like

In [31]:
display(HTML("<table><tr><td><img src='fig/pressure.png'></td><td><img src='fig/p2.png'></td></tr></table>"))

,


### Transport
And finaly, the transport solver

In [34]:
# Define transport solver
transport_solver = TransportSolver(mesh)

# Discretize and perform time stepping.
# The time stepper discretizes the advective term using a 
# first order upwind approach, while the conductive term is treated
# by TPFA.
# We saw above how the linear solver was defined in this case.
# To save each time step we send in the key-argument save_as, which
# will save the solution in a file "results/solution.pvd" as the variable
# "transport".
sol = transport_solver.solve(save_as='transport')


2018-03-19 14:38:09,893 - porepy.numerics.time_stepper - Time stepping using 10 steps
2018-03-19 14:38:09,977 - porepy.numerics.time_stepper - Step 0 out of 10


/home/eke001/anaconda3/envs/porepy/lib/python3.5/site-packages/vtk/util/numpy_support.py:134: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


2018-03-19 14:38:10,406 - porepy.numerics.time_stepper - Step 1 out of 10
2018-03-19 14:38:10,824 - porepy.numerics.time_stepper - Step 2 out of 10
2018-03-19 14:38:11,371 - porepy.numerics.time_stepper - Step 3 out of 10
2018-03-19 14:38:11,786 - porepy.numerics.time_stepper - Step 4 out of 10
2018-03-19 14:38:12,202 - porepy.numerics.time_stepper - Step 5 out of 10
2018-03-19 14:38:12,721 - porepy.numerics.time_stepper - Step 6 out of 10
2018-03-19 14:38:13,147 - porepy.numerics.time_stepper - Step 7 out of 10
2018-03-19 14:38:13,613 - porepy.numerics.time_stepper - Step 8 out of 10
2018-03-19 14:38:14,106 - porepy.numerics.time_stepper - Step 9 out of 10


And we are done. Visualized, the transport solution evolves like this

In [ ]:
HTML('<img src="fig/out.gif">')

## Comments and outlook
The notebook turned out to be rather lengthy, but if we strip away all more or less helpful comments, the following steps remains:
1. Define computational mesh.
2. Define all parameters necessary for the solution. By default, these will typically be of unit size; if other values are needed they must be overwritten. PorePy also have a convenient way of differing between parameter assignment for the rock matrix and (individual) fractures.
3. Implement a discretization strategy. PorePy defines standard solvers for pressure and transport problems, using simple but robust discretization techniques. The standard models can be modified when needed, typically in the case where the user knows something about the structure of the problem at hand.
4. Solve the problems, save relevant data.

Needless to say, these steps are part of any solution strategy.

## Possible extensions
As the intention of the example was to illustrate multi-physics couplings in the setting of flow and transport, we comment on two natural extensions:
1. Account for fluid compressibility in the flow field. Assuming the fluid is water, this can be modeled by replacing the EllipticModel with a SlightlyCompressibleModel. As the flow field is now time dependent, it should be updated during the transport solve. Exactly what is the most efficient approach there depends on the coupling strength - operator splitting is a natural option. Anyhow, it would be necessary to explictly write out the time loop containing pressure and transport solves. Apart from this most of the script should be the same.
2. As an extension of the first point, in certain applications (e.g. geothermal energy) in may be of interest to account for more ellaborate couplings, such as temperature dependency of the viscosity etc. This will in general introduce non-linear couplings that, if strong, should be solved simultaneously. This is beyond the current scope of PorePy, but should be feasible by exploiting free python libraries for automatic differentiation.

Another, more technical, step is to assign heterogeneous parameter values, mostly for cell-values fields such as permeabilities. This would require functions to define parameters e.g. based on cell center coordinates. Once this is in place, it is simply a matter of overwriting the relevant functions in data assigners such as MatrixPressureDomain etc.